# PCA Algorithm to distinguish different hand signs
By Mailin Brandt and Finian landes

In [2]:
### Imports
import numpy as np
from PIL import Image
import os

In [3]:
def load_images(dir: str, resolution: int = 256, grayscale: bool = True) -> np.ndarray:
    image_names: list = sorted(os.listdir(dir))
    n: int = len(image_names)
    images: np.ndarray = np.zeros([n, resolution ** 2])
    for i, name in enumerate(image_names):
        img: Image = Image.open(dir + "//"+ name)
        images[i] = img.getdata(0)
    return images, np.array(image_names)

def get_train_test_dataset(images: np.ndarray, image_names: list[str], n_test) -> list[np.ndarray]:
    indices: list = np.random.choice(len(images), n_test, replace = False)
    test: np.ndarray = images[indices]
    names_test: list[str] = image_names[indices]
    train: np.ndarray = np.delete(images, indices, axis = 0)
    names_train: list[str] = np.delete(image_names, indices)
    return train, test, names_train, names_test

def closest_neighbour(p: np.ndarray, m: np.ndarray , U: np.ndarray, B: np.ndarray, C: np.ndarray) -> int:
    c1: np.ndarray = np.matmul(U, p-m)
    d: np.ndarray = np.sum((c1 - C.T)**2, axis=1)
    return np.argmin(d)

def coeff_matrix(m: np.ndarray, U: np.ndarray, B: np.ndarray) -> np.ndarray:
    return np.matmul(U, (B-m).T)


In [14]:
image_dir: str = "c:/Users/finia/OneDrive - SBL/PrA/PCA of hand signs/Processed Images"
n_test_images: int = 3
images, image_names= load_images(image_dir)
train, test, names_train, names_test = get_train_test_dataset(images, image_names, n_test_images)

meanface: np.ndarray = np.mean(train, axis=0)
A: np.ndarray = train - meanface
U, s, VT = np.linalg.svd(A.transpose(), full_matrices=False)
U: np.ndarray = U.transpose()

C: np.ndarray = coeff_matrix(meanface, U, train)
successes: int = 0
for j in range(len(test)):
    p: np.ndarray = test[j]
    i: int = closest_neighbour(p, meanface, U, train, C)
    pred_name: str = names_train[i][:-7] #Removes image name suffix _XX.jpg
    real_name: str = names_test[j][:-7]
    successes += (real_name == pred_name)

s: int = successes/len(test)
print ("Success rate: "+str(np.round(s * 100, 2)) +"%")
print ("By comparison, an algorithm that only guessed would have a success rate of "+str(np.round(1 / n_test_images * 100, 2))+"%")

Success rate: 100.0%
By comparison, an algorithm that only guessed would have a success rate of 33.33%
